# 🌱 Puviyan Soil Detection - Enhanced Training (v5.0)

## 🚀 Key Features
- **GPU-accelerated** training (up to 10x faster)
- **Multiple dataset options**: Synthetic, Real, or Hybrid
- **Enhanced model architecture** for better accuracy
- **TFLite export** for Flutter deployment

In [ ]:
# @title 🚀 Setup and Configuration
print("Setting up environment...")
!nvidia-smi  # Check GPU status
!pip install -q tensorflow tensorflow-model-optimization matplotlib numpy tqdm

import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import time
from google.colab import files
from IPython.display import clear_output

# Configuration
CONFIG = {
    'model_name': 'soil_classifier_enhanced_v5',
    'input_size': 224,
    'num_classes': 8,
    'batch_size': 32,  # Will be adjusted based on GPU memory
    'epochs': 100,
    'learning_rate': 0.0005,
    'patience': 15,
    'min_delta': 1e-4
}

# Soil type labels
SOIL_LABELS = {
    0: "Alluvial Soil",
    1: "Black Soil",
    2: "Red Soil",
    3: "Laterite Soil",
    4: "Desert Soil",
    5: "Saline/Alkaline Soil",
    6: "Peaty/Marshy Soil",
    7: "Forest/Hill Soil"
}

# Setup GPU
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f\"✅ GPU Detected: {tf.test.gpu_device_name()}\")
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
else:
    print(\"⚠️ No GPU detected. Training will be very slow!\")
    print(\"   Go to Runtime > Change runtime type > Select GPU\")

## 📥 Data Loading
Run the cell below to set up data loading functions.

In [ ]:
# @title 🔄 Data Loading Functions
def generate_synthetic_data(num_samples=1000):
    print(f\"🎨 Generating {num_samples} synthetic soil samples...\")
    # Generate random images (replace with actual synthetic data generation)
    X = np.random.randint(0, 255, (num_samples, CONFIG['input_size'], CONFIG['input_size'], 3), dtype=np.uint8)
    y = np.random.randint(0, CONFIG['num_classes'], num_samples, dtype=np.int32)
    return X, y

def upload_real_data():
    from google.colab import files
    import zipfile
    
    print(\"📤 Please upload your dataset zip file\")
    uploaded = files.upload()
    
    if not uploaded:
        print(\"⚠️ No files uploaded. Using synthetic data.\")
        return None, None
    
    zip_name = list(uploaded.keys())[0]
    with zipfile.ZipFile(zip_name, 'r') as zip_ref:
        zip_ref.extractall('dataset')
    # For demo - replace with actual data loading
    print(\"📂 Loading real data...\")
    # [Add your data loading code here]
    return None, None

## 🏗️ Model Architecture
Run the cell below to define the enhanced model.

In [ ]:
# @title 🧠 Enhanced Model Architecture
def create_enhanced_model():
    print(\"🏗️ Creating enhanced model architecture...\")
    
    inputs = keras.Input(shape=(CONFIG['input_size'], CONFIG['input_size'], 3))
    
    # Data augmentation
    x = layers.Rescaling(1./255)(inputs)
    x = layers.RandomFlip(\"horizontal\")(x)
    x = layers.RandomRotation(0.2)(x)
    x = layers.RandomZoom(0.2)(x)
    
    # Feature extraction
    x = layers.Conv2D(32, 3, activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D()(x)
    
    x = layers.SeparableConv2D(64, 3, activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D()(x)
    
    x = layers.SeparableConv2D(128, 3, activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.GlobalAveragePooling2D()(x)
    
    # Classification head
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(CONFIG['num_classes'], activation='softmax')(x)
    
    model = keras.Model(inputs=inputs, outputs=outputs)
    
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=CONFIG['learning_rate']),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

## 🚀 Training
Run the cell below to start training with your selected dataset.

In [ ]:
# @title 🏃 Start Training
def train():  
    # Select dataset type  
    print(\"📊 Select dataset type:\")
    print(\"1. Synthetic data (generated)\")
    print(\"2. Upload real data\")
    print(\"3. Hybrid (synthetic + real)\")
    
    choice = input(\"Enter your choice (1-3): \").strip()
    
    # Load data based on choice  
    if choice == '1':  
        X, y = generate_synthetic_data(num_samples=5000)  
    elif choice == '2':  
        X, y = upload_real_data()  
        if X is None:  
            print(\"⚠️ Using synthetic data as fallback\")
            X, y = generate_synthetic_data(num_samples=5000)  
    elif choice == '3':  
        X_syn, y_syn = generate_synthetic_data(num_samples=2500)  
        X_real, y_real = upload_real_data()  
        if X_real is not None:  
            X = np.concatenate([X_syn, X_real], axis=0)  
            y = np.concatenate([y_syn, y_real], axis=0)  
        else:  
            print(\"⚠️ Using synthetic data only\")
            X, y = X_syn, y_syn  
    else:  
        print(\"❌ Invalid choice. Using synthetic data by default.\")
        X, y = generate_synthetic_data(num_samples=5000)  
    
    # Create and train model  
    model = create_enhanced_model()  
    
    # Callbacks  
    callbacks = [  
        keras.callbacks.EarlyStopping(  
            monitor='val_accuracy',  
            patience=CONFIG['patience'],  
            restore_best_weights=True  
        ),  
        keras.callbacks.ModelCheckpoint(  
            f\"{CONFIG['model_name']}.h5\",  
            save_best_only=True,  
            monitor='val_accuracy'  
        )  
    ]  
    
    # Train  
    print(\"\n🚀 Starting training...\")
    history = model.fit(  
        X, y,  
        validation_split=0.2,  
        batch_size=CONFIG['batch_size'],  
        epochs=CONFIG['epochs'],  
        callbacks=callbacks,  
        verbose=1  
    )  
    
    # Save model to TFLite  
    converter = tf.lite.TFLiteConverter.from_keras_model(model)  
    tflite_model = converter.convert()  
    
    with open(f\"{CONFIG['model_name']}.tflite\", 'wb') as f:  
        f.write(tflite_model)  
    
    print(f\"✅ Model saved as {CONFIG['model_name']}.tflite\")
    
    # Download the model  
    files.download(f\"{CONFIG['model_name']}.tflite\")

# Start training  
train()